# 8-Slot Full Nerdle Solver Test
This is a much larger problem, so we don't optimize the initial guess and use instead the heuristic of having many different symbols. We stick to `9*8-7=65`which needs at most $X$ guesses and $X \pm X$ guesses.

In [1]:
%load_ext autoreload
%autoreload 2

import collections
import io
import itertools
import numpy as np
import matplotlib.pyplot as plt
import sys

import nerdle
import score as s
import generator
from nerdle import Hint, NerdleData

In [6]:
# Mini-Nerdle.
NUM_SLOTS = 8
SCORE_DB_FILE = "db/nerdle{}.db".format(NUM_SLOTS) 

%time solver_data = nerdle.create_solver_data(NUM_SLOTS, SCORE_DB_FILE)

CPU times: user 8.74 ms, sys: 1.62 s, total: 1.63 s
Wall time: 1.92 s


In [7]:
d = solver_data.score_db
d.shape

(17723, 17723)

## Example Usage

In [8]:
score = nerdle.score_guess("10-43=66", "12+34=56")
print(score, 
      s.score_to_hint_string(score, NUM_SLOTS),
      s.score_to_hints(score, NUM_SLOTS) == [Hint.CORRECT, Hint.INCORRECT, Hint.INCORRECT, Hint.MISPLACED,
                               Hint.MISPLACED, Hint.CORRECT, Hint.INCORRECT, Hint.CORRECT]
     )

AttributeError: module 'nerdle' has no attribute 'score_guess'

In [9]:
# A good initial guess significantly reduces the number of answers. In this case, from
# 206 to 10.
solver = nerdle.NerdleSolver(solver_data)
guess_history, hint_history, answer_size_history = solver.solve("5+6+6=17", initial_guess= "9*8-7=65", debug=True)

--> guess 9*8-7=65 guesses_left 5
score ----?+?? 42496
answers 71
--> guess 11+46=57 guesses_left 4
score ?-?-++?+ 25890
answers 2
--> guess 5+6+6=17 guesses_left 3
score ++++++++ 21845


In [10]:
# Nerdle of September 14, 2022.
# A good initial guess significantly reduces the number of answers. In this case, from 20177236 to 70.
solver = nerdle.NerdleSolver(solver_data)
#hint_generator = s.FileHintGenerator(sys.stdin)
hints = [
    "-----++?", 
    "+-+?-+--",
    "++++++++",
]
hint_generator = s.FileHintGenerator(io.StringIO("\n".join(hints)))
%time guess_history, hint_history, answer_size_history = solver.solve_adversary(hint_generator.__call__, initial_guess= "9*8-7=65", debug=True)

--> guess 9*8-7=65 guesses_left 5
score -----++? 37888
answers 70
--> guess 11+23=34 guesses_left 4
score +-+?-+-- 1169
answers 2
--> guess 10+52=62 guesses_left 3
score ++++++++ 21845
CPU times: user 494 ms, sys: 16.4 ms, total: 511 ms
Wall time: 510 ms


## Benchmark
This is a fast in-memory dict implementation.

In [11]:
# Nerdle of September 14, 2022.
solver = nerdle.NerdleSolver(solver_data)
%time guess_history, hint_history, answer_size_history = solver.solve("10+52=62", initial_guess="9*8-7=65")

CPU times: user 623 ms, sys: 5.17 ms, total: 628 ms
Wall time: 635 ms


## Performance -- # Guesses Distribution
Assuming a start with a lot of different numbers and operators is best. Let's try all possible answers and observe how many guesses wew need on average. In fact, if we do well with the initial guess, we could attempt to build a lookup table of guesses vs. hints, and just use this static table; although recalculation takes just 0.5 seconds now!

In [43]:
start = "9*8-7=65"
solutions = [nerdle.NerdleSolver(solver_data).solve(answer, initial_guess=start) 
             for answer in solver_data.answers]               
n = np.array([len(solution[0]) for solution in solutions])
num_answers = len(solver_data.answers)
compression_ratio = num_answers / np.array([solution[2][0] for solution in solutions])
print(np.mean(compression_ratio), np.std(compression_ratio))

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

ax = axs[0]
ax.hist(n);
ax.set_title("#Guesses for start {}".format(start));

ax = axs[1]
ax.hist(compression_ratio);
ax.set_title("Compression Ratio Distribution, start {}".format(start));

KeyboardInterrupt: 